# Text Generation using LSTMs on the articles dataset

In [40]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)
import os,string
import warnings
warnings.filterwarnings("ignore")
import numpy as np 

In [4]:
# load the data 
data = pd.read_csv("data_with_labels.csv")
data.head()

,Unnamed: 0,title,date,SubTitle,site,dominant_topic,topic_labels,cluster
0,0,mitigating ethical risks in generative ai stra...,2024-01-03 00:00:00.000,artificial intelligence ai has been around for...,DSC,6,AI and Models,4
1,1,dsc weekly january,2024-01-02 00:00:00.000,announcements top stories in depth,DSC,7,Comprehensive Guide,6
2,2,mastering iot data management for business suc...,2024-01-01 00:00:00.000,in today s tech driven landscape the prolifera...,DSC,6,AI and Models,4
3,3,generative ai business model disruption the ny...,2024-01-01 00:00:00.000,will be all about changing business models due...,DSC,6,AI and Models,4
4,4,genai synthesizing dna sequences with llm tech...,2024-01-01 00:00:00.000,when people talk about large language models t...,DSC,8,Coding and Implementation,0


In [5]:
data['text'] = data['title']+data['SubTitle']
data.head()

,Unnamed: 0,title,date,SubTitle,site,dominant_topic,topic_labels,cluster,text
0,0,mitigating ethical risks in generative ai stra...,2024-01-03 00:00:00.000,artificial intelligence ai has been around for...,DSC,6,AI and Models,4,mitigating ethical risks in generative ai stra...
1,1,dsc weekly january,2024-01-02 00:00:00.000,announcements top stories in depth,DSC,7,Comprehensive Guide,6,dsc weekly januaryannouncements top stories in...
2,2,mastering iot data management for business suc...,2024-01-01 00:00:00.000,in today s tech driven landscape the prolifera...,DSC,6,AI and Models,4,mastering iot data management for business suc...
3,3,generative ai business model disruption the ny...,2024-01-01 00:00:00.000,will be all about changing business models due...,DSC,6,AI and Models,4,generative ai business model disruption the ny...
4,4,genai synthesizing dna sequences with llm tech...,2024-01-01 00:00:00.000,when people talk about large language models t...,DSC,8,Coding and Implementation,0,genai synthesizing dna sequences with llm tech...


In [8]:
STOPWORDS = set(stopwords.words('english'))

In [9]:
# data prep
def simple_clean_text(x):
    x = x.lower()
    x = re.sub('[^a-z]'," ",x)
    x = re.sub(' +',' ',x).strip()
    words = x.split()
    
    lemmatizer = WordNetLemmatizer()
    words =[lemmatizer.lemmatize(i) for i in words]

    words = [w for w in words if w not in STOPWORDS]
        
        
    return " ".join(words)

In [10]:
data['text'] = data['text'].apply(lambda x:simple_clean_text(x))
data.text.head()

0    mitigating ethical risk generative ai strategy...
1      dsc weekly januaryannouncements top story depth
2    mastering iot data management business success...
3    generative ai business model disruption nyt la...
4    genai synthesizing dna sequence llm techniques...
Name: text, dtype: object

In [11]:
corpus = [x for x in data.text]
corpus[:10]

['mitigating ethical risk generative ai strategy safe secure ai applicationartificial intelligence ai ha around many decade ha become buzzword even among non technical people generative ai read',
 'dsc weekly januaryannouncements top story depth',
 'mastering iot data management business successin today tech driven landscape proliferation internet thing iot device ha revolutionized business collect utilize data interconnectivity device read',
 'generative ai business model disruption nyt lawsuit posturingwill changing business model due massive disruption generative ai new winner many loser read',
 'genai synthesizing dna sequence llm techniqueswhen people talk large language model common topic text summarization text generation answering prompt gpt yet read',
 'genai beware productivity trap economics partit technology advancement game changer game changer technological advancement leveraged economically transform industry society read',
 'best kept secret llmsgpt great tool write sum

In [28]:
tokenizer = Tokenizer()

def get_the_seq(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index)+1

    input_seq=[]

    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1,len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_seq.append(n_gram_sequence)
    
    return input_seq, total_words

In [29]:
input_sequences, total_words = get_the_seq(corpus)

In [30]:
input_sequences[:10]

[[738, 739],
 [738, 739, 740],
 [738, 739, 740, 24],
 [738, 739, 740, 24, 2],
 [738, 739, 740, 24, 2, 38],
 [738, 739, 740, 24, 2, 38, 741],
 [738, 739, 740, 24, 2, 38, 741, 274],
 [738, 739, 740, 24, 2, 38, 741, 274, 2],
 [738, 739, 740, 24, 2, 38, 741, 274, 2, 742],
 [738, 739, 740, 24, 2, 38, 741, 274, 2, 742, 73]]

In [33]:
import tensorflow.keras.utils as ku

In [34]:
def generated_padded_sequence(input_sequences):
    max_sequences_len = max([len(x) for x in input_sequences])

    input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequences_len,padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label,num_classes=total_words)
    return predictors, label, max_sequences_len

In [35]:
predictors,label,max_seq_len = generated_padded_sequence(input_sequences)

In [39]:
print(max_seq_len)

27


In [41]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_seq_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 26, 10)            22790     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2279)              230179    
                                                                 
Total params: 297369 (1.13 MB)
Trainable params: 297369 (1.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [51]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        print(predicted)
        predicted_class = np.argmax(predicted,axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted_class:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [54]:
print (generate_text("AI and LLM", 5, model, max_seq_len))

[[7.1145201e-10 2.9996701e-05 5.7553199e-05 ... 1.2663775e-04
  1.4239479e-08 4.3614312e-09]]
[[3.2745223e-10 8.7946793e-03 2.7717939e-02 ... 1.0498176e-09
  4.3125264e-11 1.1307131e-09]]
[[3.5288672e-10 1.5929488e-02 1.4645747e-02 ... 6.0002718e-11
  2.9633191e-09 2.7785980e-09]]
[[1.0726179e-09 8.9120970e-04 3.9054780e-06 ... 7.4162747e-09
  6.2736788e-08 2.6878329e-08]]
[[2.07726805e-11 1.50682395e-02 2.92730797e-03 ... 1.71197916e-15
  3.89748184e-13 4.60030104e-12]]
Ai And Llm Diffusion Scratch Using Pythondeveloping Learning
